In [ ]:
# Set environment variables
export REGION=us-west1
export ZONE=us-west1-a
export PROJECT_ID=qwiklabs-gcp-04-849b57aab730

##Task 1. Create a Kubernetes cluster

In [ ]:
# Your test environment is limited in capacity, so
# you should limit the test Kubernetes cluster you are creating to just two e2-standard-2 instances.
# You must call your cluster echo-cluster.

gcloud container clusters create echo-cluster \
    --num-nodes=1 \
    --zone=$ZONE \
    --machine-type=e2-standard-2 \
    --enable-ip-alias

In [ ]:
gcloud container clusters list

##Task 2. Build a tagged Docker image

In [ ]:
# The sample application, including the Dockerfile and the application context files, are contained in an archive called echo-web.tar.gz.
# The archive has been copied to a Cloud Storage bucket belonging to your lab project called gs://[PROJECT_ID].

You must deploy this with a tag called v1.

gsutil cp gs://${PROJECT_ID}/echo-web.tar.gz .

In [ ]:
tar -xvzf echo-web.tar.gz

In [ ]:
docker build -t echo-app:v1 .

In [ ]:
docker images

##Task 3. Push the image to the Google Container Registry

In [ ]:
# Your organization has decided that it will always use the gcr.io Container Registry hostname for all projects.
# The sample application is a simple web application that reports some data describing the configuration of the system where the application is running.
# It is configured to use TCP port 8000 by default.

docker tag echo-app:v1 gcr.io/${PROJECT_ID}/echo-app:v1

In [ ]:
docker push gcr.io/${PROJECT_ID}/echo-app:v1

In [ ]:
gcloud container images list --repository=gcr.io/${PROJECT_ID}

##Task 4. Deploy the application to the Kubernetes cluster

In [ ]:
# Even though the application is configured to respond to HTTP requests on port 8000,
# you must configure the service to respond to normal web requests on port 80.
# When configuring the cluster for your sample application, call your deployment echo-web.

kubectl create deployment echo-web --image=gcr.io/${PROJECT_ID}/echo-app:v1

In [ ]:
gcloud container clusters get-credentials echo-cluster --zone=$ZONE

In [ ]:
kubectl expose deployment echo-web \
    --type=LoadBalancer \
    --port=80 \
    --target-port=8000

In [ ]:
kubectl get service echo-web